<a href="https://colab.research.google.com/github/paulanavarretec/WineRec/blob/master/wineRec_OneHotEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
# Example dummy data from Rendle 2010 
# http://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf
# Variables categoricas (Users, Wines, Last Rated) pasaron por one-hot-encoding 

x_data = np.matrix([
#     Users  |     Wines      |    Wine Ratings    | Time | Last Wines Rated
#    A  B  C | W1  W2  W3  W4 | W1   W2   W3   W4  |      | W1  W2  W3  W3
    [1, 0, 0,  1,  0,  0,  0,   0.3, 0.3, 0.3, 0,     13,   0,  0,  0,  0 ],
    [1, 0, 0,  0,  1,  0,  0,   0.3, 0.3, 0.3, 0,     14,   1,  0,  0,  0 ],
    [1, 0, 0,  0,  0,  1,  0,   0.3, 0.3, 0.3, 0,     16,   0,  1,  0,  0 ],
    [0, 1, 0,  0,  0,  1,  0,   0,   0,   0.5, 0.5,   5,    0,  0,  0,  0 ],
    [0, 1, 0,  0,  0,  0,  1,   0,   0,   0.5, 0.5,   8,    0,  0,  1,  0 ],
    [0, 0, 1,  1,  0,  0,  0,   0.5, 0,   0.5, 0,     9,    0,  0,  0,  0 ],
    [0, 0, 1,  0,  0,  1,  0,   0.5, 0,   0.5, 0,     12,   1,  0,  0,  0 ]
])

# ratings
y_data = np.array([5, 3, 1, 4, 5, 1, 5])

# Agregamos un eje para que funcione Tensorflow.
y_data.shape += (1, )

import tensorflow as tf
n, p = x_data.shape

# numero de latent factors
k = 5

# design matrix
X = tf.placeholder('float', shape=[n, p])
# target vector
y = tf.placeholder('float', shape=[n, 1])

# bias y weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# interaction factors, inicializados de forma aleatoria 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimación de y, inicializada en 0.
y_hat = tf.Variable(tf.zeros([n, 1]))

Usamos placeholders para los inputs y targets. Los datos actuales se asignarán en el runtime de la sesión. X e Y no serán modificados por el backend; Usamos variables para almacenar bias, weights y factor layers. Estos son los parámetros que se actualizarán al ajustar el modelo.

En el siguiente código, computamos WX y usamos reduce_sum() para agregar las filas del Tensor resultante (axis 1). keep_dims está seteado en True para asegurar que las dimensiones input/output se respeten.

In [0]:
linear_terms = tf.add(w0,
        	tf.reduce_sum(
          tf.multiply(W, X), 1, keepdims=True))

En chunk anterior implementamos simplemente una regresion lineal. Hacemos lo mismo con los términos de interacción.

In [0]:
interactions = (tf.multiply(0.5,
                tf.reduce_sum(
                    tf.subtract(
                        tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                        tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),
                    1, keep_dims=True)))

Y agregamos todo para obtener el target estimado.

In [0]:
y_hat = tf.add(linear_terms, interactions)

Como estamos resolviendo un problema de regresion, aprendemos los parámetros del modelo minimizando la suma de los residuos cuadráticos como función de pérdida. También agregamos un término de regularización para prevenir overfitting.

In [0]:
# Función de pérdida como suma de cuadrados regularizada L2  sobre W y V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
loss = tf.add(error, l2_norm)

Para entrenar el modelo declamos un optimizador y minimizamos la función de pérdida.

In [0]:
eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

Estamos listos para obtener el grafo y desplegarlo en el backend de Tensorflow. Usamos pythos context manager pata manejar la sesion.

In [18]:
# Muchas Iteraciones
N_EPOCHS = 1000

# Desplegar grafo
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(N_EPOCHS):
        indices = np.arange(n)
        np.random.shuffle(indices)
        x_data, y_data = x_data[indices], y_data[indices]
        sess.run(optimizer, feed_dict={X: x_data, y: y_data})

    print('MSE: ', sess.run(error, feed_dict={X: x_data, y: y_data}))
    print('Loss (regularized error):', sess.run(loss, feed_dict={X: x_data, y: y_data}))
    print('Predictions:', sess.run(y_hat, feed_dict={X: x_data, y: y_data}))
    print('Learnt weights:', sess.run(W, feed_dict={X: x_data, y: y_data}))
    print('Learnt factors:', sess.run(V, feed_dict={X: x_data, y: y_data}))

MSE:  4.5705195e-07
Loss (regularized error): 0.0031154503
Predictions: [[3.0001822]
 [1.0004226]
 [4.9992223]
 [1.0009962]
 [4.9996285]
 [4.999851 ]
 [3.9988909]]
Learnt weights: [ 0.15683964  0.18582082 -0.11557835 -0.01759121 -0.05105131  0.20488892
  0.08483714 -0.0028634   0.09054252  0.07809868  0.14044799  0.11401302
  0.1677825  -0.16010605  0.08483714  0.        ]
Learnt factors: [[-0.03877404  0.09972517 -0.02663375 -0.02762474 -0.11602374  0.05021128
   0.06052936 -0.02106447 -0.03125477  0.02357763  0.09259289  0.05842411
   0.05749567 -0.18113753  0.06019074 -0.00141657]
 [ 0.15471348  0.22781421 -0.09609849  0.13426377 -0.2403505   0.20543666
   0.04828157  0.02070697  0.08827759  0.09127898  0.19516642  0.24714474
   0.24685891 -0.3831636   0.06615476 -0.00153511]
 [ 0.10873464  0.21694219 -0.14482094  0.10426582 -0.2676071   0.17748334
   0.04517152 -0.02715301  0.07373484  0.05430392  0.18379144  0.24976592
   0.20171249 -0.3793244   0.05433233 -0.01011759]
 [ 0.121261

En cada iteración (hasta N_EPOCHS) ejecutamos el optimizador, el cual recalcula los parametros del modelo por el método de descenso del gradiente. Notar que estamos moviendo datos desde el espacio de memoria de Python al de backend C++ de Tensorflow  a través de feed_dict={}.
Como estamos trabajando con un set reducido de datos exprimentales podemos entregar el dataset todo de una vez. En la práctica, querremos trabajar con mini batches (ej. usando un generador sobre el input). Podemos mezclar los datos (np.random.shuffle) para evitar sesgar el gradiente.

# Experimentos

Para setear un benchmark  del performance en general del modelo, entrenamos y testeamos en el dataset porcionado en training y test (70%-30%). El objetivo es predecir el rating para cada vino. Luego del aplicar one-hot-encoding a las variables categóricas, obtendemos una design matrix de 90570×2623 [Corregir] para el training set, y de 90570×2623[Corregir] para el test set.
